In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as st
import pandas as pd
import seaborn as sns

from collections import Counter

import nltk.lm as lm
from nltk.util import ngrams as nltk_ngrams
import numpy as np
import scipy.stats as st

In [2]:
data = pd.read_csv('sadiev.aa.csv', header = None)
data.head(5)

,0
0,0
1,1
2,0
3,1
4,0


In [3]:
X = data.values.T.reshape(data.values.T.shape[1], )
X

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

- Оценка $\mathbb{P}(X_{i+1}= 0|X_{i}=0)$ методом максимального правдоподобия

In [4]:
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

ngram_counts, prefix_counts = ngrams_and_prefix_counts(list(X), 2)

def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2

p2 = bigram_probas(ngram_counts, prefix_counts)

print('P(0|0) = 1 - P(1|0) = {}'.format(1 - p2['1|0']))

P(0|0) = 1 - P(1|0) = 0.0


- Отвергается ли нулевая гипотеза $\mathbb{P}(X_{i+1}= 0|X_{i}=0)=0.9$?

Введем статистику $$T = \sum^{n - 1}_{i = 1}I[X_{i+1} = 0, X_i = 0] $$
Нулевое распределение - биномиальное распределение  $Bin(n-1, p)$
Для нашей последовательности $T = 0$,  поскольку частота `'0|0'`, посчитанная выше, есть ноль.
Поэтому сразу видно, что гипотеза отклоняется в силу того, что $0 < Bin(n-1, p = 0.9)(\alpha/2)$

- Какова вероятность встретить $X_i=2$ с применением сглаживания Лапласа

In [6]:
voc = list(set(X))
voc.append(2)
f = []
for i in voc:
    f.append(len(X[X == i]))

p_2 = (f[2]+ 1)/(sum(f) + len(voc) )
print('вероятность встретить X_i=2: {}'.format(p_2))

вероятность встретить X_i=2: 0.02631578947368421
